# Tapis v3 Hands-on

In this notebook, you will use Tapis v3 to create two systems and an application that will be used to run
an MPM job on a HPC like VM.

To execute each `In[#]` cell, you can click inside the cell and press `Shift + Enter`

## Install the Tapipy Python SDK

In [ ]:
pip install tapipy

## Enter training account information

To get things started, please run the following and enter the training account information provided to you. The username and password will be same trainingXX/trainingXX

In [166]:
import getpass

tenant = 'training'
base_url = 'https://' + tenant + '.tapis.io'

username = input('Username: ')
password = getpass.getpass(prompt='Password: ', stream=None)


## Authenticate and initialize Tapis v3 client

Using this information, you can now use `tapipy` to authenticate in the tenant and initialize the
Tapis v3 client. You should see your token information displayed. This may take a while to run but should take
no more than 30 seconds.

In [ ]:
from tapipy.tapis import Tapis
#Create python Tapis client for user
client = Tapis(base_url= base_url, username=username, password=password)
# *** Tapis v3: Call to Tokens API
client.get_tokens()
# Print Tapis v3 token
client.access_token

In order to create Tapis Systems, we need an actual user on the VM. For simplicity we have created the same trainingXX user on the host. The password will be the alphanumeric provided to you.

In [55]:
password_vm = getpass.getpass(prompt='Password for VM: ', stream=None)
host = input('Host: ')

## Systems

In this section we create a Tapis systems, one for running on a VM host using FORK and one for running on an HPC type host using BATCH.

Note that although it is possible, we have not provided any login credentials in the system definitions.
Well-crafted system definitions are likely to be copied and re-used, so, for security reasons, it is recommended that
login credentials be registered using separate API calls as discussed below.

### Create a system for the VM host

In [ ]:
user_id = username
system_id_vm = "tapis-vm-" + user_id

# Create the system definition
exec_system_vm = {
  "id": system_id_vm,
  "description": "Test system",
  "systemType": "LINUX",
  "host": host,
  "defaultAuthnMethod": "PASSWORD",
  "rootDir": "/home/"+user_id,
  "canExec": True,
  "jobRuntimes": [ { "runtimeType": "SINGULARITY" } ],
  "jobWorkingDir": "workdir",
}

# Use the client to create the system in Tapis
print("****************************************************")
print("Create system: " + system_id_vm)
print("****************************************************")
client.systems.createSystem(**exec_system_vm)


# If you need to update the system, you can modify the original definition and use the putSystem call.
# - modify the above definition as needed
# - comment out the above line with the call to createSystem()
# - uncomment the below line with the call to updateSystem()
# - re-run the cell
# Note that not all attributes may be updated.
#client.systems.putSystem(**exec_system_vm, systemId=system_id_vm)

In [ ]:
# You can also update just a few attributes using the patchSystem call.
# Note that not all attributes may be updated and some attributes, such as *enabled*,
#   may only be updated using a specific call.
# For example, to update the description, first define the json to be used:
patch_system_vm = {
  "description": "System for testing jobs on a VM for Tapis tutorial"
}

# Then use the client to make the update:
client.systems.patchSystem(**patch_system_vm, systemId=system_id_vm)

In [ ]:
# List all systems available to you
print("****************************************************")
print("List all systems")
print("****************************************************")
client.systems.getSystems()

In [ ]:
#client.systems.deleteSystem(systemId='tapis-vm-training1')

In [ ]:
# Get details for the system you created
print("****************************************************")
print("Fetch system: " + system_id_vm)
print("****************************************************")
client.systems.getSystem(systemId=system_id_vm)

### Register Credentials for the VM system

After creating the system, you will need to register credentials for your username. These will be used by Tapis to
access the host. Various authentication methods can be used to access a system, such as PASSWORD and PKI_KEYS. For the
VM a password is used.

In [ ]:
# Register credentials
client.systems.createUserCredential(systemId=system_id_vm, userName=user_id, password=password_vm)

Now you can use the client to list files on the system. This will confirm that the credentials are valid.

In [ ]:
# List files at the rootDir for the system
client.files.listFiles(systemId=system_id_vm, path="/")

### Create a system for the HPC cluster

With just a few changes to the system definition you can create a second system that can be used to run the
same application on an HPC type host. Note the minimal changes:

* **id** - A unique id is required
* **host** - Main hostname for the HPC system.
* **rootDir** - Using the root directory of the host gives us flexibility in setting **jobWorkingDir**.
  Note that you still need LINUX permissions.
* **jobWorkingDir** - Now determined dynamically using the Tapis v3 function HOST_EVAL()
* **jobRuntimes** - Most HPC systems support singularity and not docker
* **batchLogicalQueue.hpcQueueName** - HPC queue to use by default.
* **batchLogicalQueues** - HPC queue definitions for this HPC system.

In [ ]:
user_id = username
system_id_hpc = "tapis-hpc-" + user_id 

# Create the system definition
exec_system_hpc = {
  "id": system_id_hpc,
  "description": "System for testing jobs on an HPC type host for tapis tutorial",
  "systemType": "LINUX",
  "host": host,
  "defaultAuthnMethod": "PASSWORD",
  "rootDir": "/home/"+user_id,
  "canExec": True,
  "jobRuntimes": [ { "runtimeType": "SINGULARITY" } ],
  "jobWorkingDir": "workdir",
  "canRunBatch": True,
  "batchScheduler": "SLURM",
  "batchSchedulerProfile": "tacc",
  "batchDefaultLogicalQueue": "tapisNormal",
  "batchLogicalQueues": [
    {
      "name": "tapisNormal",
      "hpcQueueName": "normal",
      "maxJobs": 50,
      "maxJobsPerUser": 10,
      "minNodeCount": 1,
      "maxNodeCount": 16,
      "minCoresPerNode": 1,
      "maxCoresPerNode": 68,
      "minMemoryMB": 1,
      "maxMemoryMB": 16384,
      "minMinutes": 1,
      "maxMinutes": 60
    }
  ]
}

# Use the client to create the system in Tapis
print("****************************************************")
print("Create system: " + system_id_hpc)
print("****************************************************")
client.systems.createSystem(**exec_system_hpc)

# If you need to update the system,
# - modify the above definition as needed
# - comment out the above line
# - uncomment the below line
# - re-run the cell
#client.systems.putSystem(**exec_system_hpc, systemId=system_id_hpc)


In [ ]:
# List all systems available to you
print("****************************************************")
print("List all systems")
print("****************************************************")
client.systems.getSystems()

In [ ]:
# Get details for the system you created
print("****************************************************")
print("Fetch system: " + system_id_hpc)
print("****************************************************")
client.systems.getSystem(systemId=system_id_hpc)

### Register Credentials for the HPC system

As before, now you will need to register credentials for your username. These will be used by Tapis to
access the host.

In [ ]:
password_hpc = password_vm
# Register credentials
client.systems.createUserCredential(systemId=system_id_hpc, userName=user_id, password=password_hpc)

Now you can use the client to list files on the system. This will confirm that the credentials are valid.

In [ ]:
# List files at the rootDir for the system
path_to_list = "/"
client.files.listFiles(systemId=system_id_hpc, path=path_to_list)

## Application

In order to run a job on a system you will need to create a Tapis application.

### Create an application that can be run on the VM host or the HPC cluster

In [ ]:
user_id = username
app_id = "mpm-docker-" + user_id

# Create the application definition
app_def = {
    "id": app_id,
    "version": "dev",
    "jobType": "FORK",
    "runtime": "DOCKER",
    "description": "High-Performance Material Point Method (CB-Geo mpm) DEVELOPMENT version.",
    "containerImage": "tapis/mpm:dev",
    "jobAttributes": {
        "isMpi": False,
        "parameterSet": {
            "appArgs": [
                {"name": "directoryInputFlag", "arg": "-f", "inputMode": "FIXED"},
                {"name": "directoryInput", "arg": "/home/cbgeo/research/mpm-benchmarks/2d/uniaxial_stress/", "inputMode": "REQUIRED"}
            ] 
        },
        "fileInputs": [
            {
                "name": "directoryInput",
                "inputMode": "OPTIONAL",
                "targetPath": ".",
                "description": "Input directory that contains the MPM congiguration file as well as any other required files. Note that to utilize this attribute one must also set the directoryInput parameter to mbe the value of the name of the directory. Also note that if this directory is not provided, a default (included in the appliation container image) will be used."
            }
        ]
    }

}

# Use the client to create the application in Tapis
print("****************************************************")
print("Create application: " + app_id)
print("****************************************************")
client.apps.createAppVersion(**app_def)

# If you need to update the application,
# - modify the above definition as needed
# - comment out the above line
# - uncomment the below line
# - re-run the cell
#client.apps.putApp(**app_def, appId=app_id, appVersion="0.0.1")

In [ ]:
# List all applications available to you
print("****************************************************")
print("List all applications")
print("****************************************************")
client.apps.getApps()

In [ ]:
# Get details for the application you created
print("****************************************************")
print("Fetch application: " + app_id)
print("****************************************************")
client.apps.getAppLatestVersion(appId=app_id)

## Jobs

We will run two jobs, one on the VM host using FORK and one on the HPC type host using BATCH.

We will use the same Tapis application to run both jobs.

### Part 1: Run Material Point Method (MPM) app on a Virtual Machine.


In [65]:
# Run MPM app on a Virtual Machine

# Submit a job
job_response_vm=client.jobs.submitJob(name='mpm-job-vm',description='material point method',appId=app_id,execSystemId=system_id_vm,appVersion= 'dev')

### Get Job submission response


In [ ]:
# Get Job submission response
print("****************************************************")
print("Job Submitted: " + app_id)
print("****************************************************")
print(job_response_vm)

### Get Jobs Listings


In [ ]:
# Get Jobs listings
client.jobs.getJobList()

### Get Job UUID from the submission response


In [ ]:
# Get job uuid from the job submission response
print("****************************************************")
job_uuid_vm=job_response_vm.uuid
print("Job UUID: " + job_uuid_vm)
print("****************************************************")

### Check the status of the job


In [ ]:
# Check the status of the job
print("****************************************************")
print(client.jobs.getJobStatus(jobUuid=job_uuid_vm))
print("****************************************************")

### Download output of the job


In [ ]:
# Once the job is in the FINISHED state, you can download output of the job
print("Job Output file:")

print("****************************************************")
jobs_output_vm= client.jobs.getJobOutputDownload(jobUuid=job_uuid_vm,outputPath='stdout')
print(jobs_output_vm)
print("****************************************************")

### Cancel a job


In [ ]:
# If necessary, you can cancel a long running job.
# To cancel a running job
# client.jobs.cancelJob(jobUuid=job_uuid_vm)

## Part 2: Run a Batch Job on HPC type host

Using the same Tapis application we can also run the image classifier as a batch job on an HPC type host


In [87]:
# Run MPM app on the HPC Machine

# Submit a job
job_response_hpc=client.jobs.submitJob(name='mpm-hpc',description='mpm',appId=app_id,execSystemId=system_id_hpc,appVersion= 'dev')

### Get Job submission response


In [ ]:
print("****************************************************")
print("Job Submitted: " + app_id)
print("****************************************************")
print(job_response_hpc)

### Check job status


In [ ]:
# Check the status of the job
print("****************************************************")
job_uuid_hpc=job_response_hpc.uuid
print(client.jobs.getJobStatus(jobUuid=job_uuid_hpc))
print("****************************************************")

### Download output of the HPC job


In [ ]:
# Download output of the job
print("Job Output file:")

print("****************************************************")
jobs_output_hpc= client.jobs.getJobOutputDownload(jobUuid=job_uuid_hpc,outputPath='stdout')
print(jobs_output_hpc)
print("****************************************************")

## Workflows

In this section, we are going to use tapipy to construct a pipeline that builds and HPC application container image, pushes it to a remote image registry, then run some tests in a container using the HPC application 

### Dockerhub Credentials

First we need to set our Dockerhub credentials. This will be used to give the image builder permissions to push to your Dockerhub account.

#### NOTE:
Your Dockerhub credentials will be encrypted and safely stored in the Tapis Security Kernel (backed by HasiCorp Vault)

In [50]:
dockerhub_username = input('Dockerhub username: ')
dockerhub_personal_access_token = getpass.getpass(prompt='Dockerhub Access Token: ', stream=None)

### Create a Group
All workflow resources must exist within a group. A group is collection of users that have access to workflow resources such as Pipelines and Tasks. Anyone that belongs to a group can create their own pipelines and run pipelines owned by that group.

In [ ]:
# Create the group
group_id = "pearc23-group-" + username

print("****************************************************")
create_group_resp = client.workflows.createGroup(id=group_id)
print(create_group_resp)
print("****************************************************")

### Create a Pipeline
Pipelines are simply collections of tasks. Tasks can be added to a pipeline after it is created or directly in the pipeline definition itself. For this demonstration we will be creating everything at once.

The first task in this pipeline is an image build task. Image build tasks require a "context", which is the source control repository which contains the Dockerfile we want to build from.

The next two tasks run jobs on an HPC system to ensure that there are no errors with the image. The first test ensures that MPM was compiled correctly and the second run a test script called uniaxial traction

In [ ]:
# Create the group
pipeline_id = "pearc23-pipeline-" + username

print("****************************************************")
create_pipeline_resp = client.workflows.createPipeline(**{
    "id": pipeline_id,
    "group_id": group_id,
    "type": "workflow",
    "execution_profile": {
        "max_retries": 0,
        "invocation_mode": "async",
        "duplicate_submission_policy": "terminate", # Terminates the current running pipeline if another is submitted
        "max_exec_time": 3600 # in seconds
    },
    "tasks": [
        {
            "id": "build-mpm-image",
            "pipeline_id": pipeline_id,
            "group_id": group_id,
            "type": "image_build",
            "builder": "kaniko", # Alternative to docker that allows you to build containers in containers
            "context": {
                "type": "github",
                "branch": "main",
                "url": "tapis-project/application-repository",
                "build_file_path": "Dockerfile",
                "sub_path": "/material-point-method/mpm-dummy-src/docker_build",
                "visibility": "public"
            },
            "destination": {
                "type": "dockerhub",
                "url": f"{dockerhub_username}/dummy-mpm",
                "tag": "pearc-test",
                "credentials": {
                    "username": dockerhub_username,
                    "token": dockerhub_personal_access_token
                }
            }
        },
        {
            "id": "test-mpm-compiled",
            "type": "tapis_job",
            "tapis_job_def": {
                "name": 'mpm-compiled-correctly',
                "description": 'material point method',
                "appId": app_id,
                "execSystemId": system_id_vm,
                "appVersion": 'dev'
            },
            "depends_on": [
                {"id": "build-mpm-image"}
            ]
        },
        {
            "id": "test-mpm-uniaxial-traction",
            "type": "tapis_job",
            "tapis_job_def": {
                "name": "mpm-uniaxial-traction-test",
                "appId": app_id,
                "appVersion": "dev",
                "execSystemId": system_id_vm,
                "appArgs": {
                    "directoryInput": "./benchmarks/2d/uniaxial_traction/"
                }
            },
            "depends_on": [
                {"id": "build-mpm-image"}
            ]
        }
    ]
})
print(create_pipeline_resp)
print("****************************************************")

### Running a pipeline

Once a pipeline has been definined it can now be run with a simple call the runPipeline endpoint.

#### NOTE:

In our execution profile of our pipeline definition, we set the execution profile to `terminate` this means that once you run a pipeline, all subsequent submissions of a pipeline to workflow engine will terminate the one previously running, so only run the cell below a single time or the next run of the pipeline may end up delayed.


In [ ]:
print("****************************************************")
runs = client.workflows.listPipelineRuns(group_id=group_id, pipeline_id=pipeline_id)
sorted_runs = sorted(runs, key=lambda run: run.started_at, reverse=True)
if (
    len(sorted_runs) == 0 
    or (len(sorted_runs) > 0 and sorted_runs[0].status not in ["pending", "active"])
):
    run_pipeline_resp = client.workflows.runPipeline(group_id=group_id, pipeline_id=pipeline_id)
    print(run_pipeline_resp)
else:
    print(f"Pipeline currently {sorted_runs[0].status}")
print("****************************************************")

### Checking the PipelineRun Status

Your pipeline is now running. It will take some time for the HPC image to build. In the meantime, you can check the status of the run by running the cell below

In [ ]:
pipeline_runs = client.workflows.listPipelineRuns(group_id=group_id, pipeline_id=pipeline_id)
last = sorted(pipeline_runs, key=lambda run: run.started_at, reverse=True)[0]
task_executions = client.workflows.listTaskExecutions(group_id=group_id, pipeline_id=pipeline_id, pipeline_run_uuid=last.uuid)
print("****************************************************")
print(f"Current Pipeline Run - {last.status}")
print("****************************************************")
print(f"Task Executions [{len(task_executions)}]")
print("****************************************************")
for i, execution in enumerate(sorted(task_executions, key=lambda ex: ex.started_at)):
    print(f"[{i}]", execution.task_id, execution.status)
print("****************************************************")